In [1]:
import math
import os
import cv2
import shutil
import numpy as np
import pandas as pd

from osgeo import gdal, osr
from tqdm import tqdm

In [76]:
SOURCE = '/peta_rename'
TEMP_ROI = '/Terry/georeference/temp_file/ROI'
TEMP_CROP = '/Terry/georeference/temp_file/Crop'
COORD = '/Terry/georeference/sls_st_coordinates.csv'
RESULT = 'georeferenced_maps_export'

In [70]:
def get_loc(input_path, output_path):
    x = 50
    y = 100

    # Load the image
    image = cv2.imread(input_path)

    height, width = image.shape[:2]

    if width > height:
        width = round(2650/3306*width)
        height = round(2300/2337*height)

    else:
        width = round(2300/2337*width)
        height = round(2650/3306*height)

    # Crop the image using the provided coordinates and dimensions
    cropped_image = image[y:y+height, x:x+width]

    # Save the cropped image
    cv2.imwrite(output_path, cropped_image)

In [106]:
def crop_map(input_path, output_path):
    # Load the JPEG file
    img = cv2.imread(input_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply a Canny edge detection filter to find the edges
    edges = cv2.Canny(gray, 50, 150)

    # Find contours in the image
    contours, hierarchy = cv2.findContours(
        edges, 
        cv2.RETR_EXTERNAL, 
        cv2.CHAIN_APPROX_SIMPLE)

    # Initialize variables to track the largest contour and its area
    largest_contour = None
    largest_area = 0

    # Iterate through the detected contours to find the largest one
    for contour in contours:
        contour_area = cv2.contourArea(contour)
        if contour_area > largest_area:
            largest_area = contour_area
            largest_contour = contour
    
    height, width = image.shape[:2]
            
    if width > height:
        min_size_threshold = (
            (2000/2300*height)
            * (2000/2650*width)
        )
    else:
        min_size_threshold = (
            (2000/2300*width)
            * (2000/2650*height)
        )
    
    # Check if the largest contour meets the size threshold
    if largest_contour is not None and largest_area >= min_size_threshold:
        # Get the bounding box coordinates
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Extract the image inside the bounding box
        img_cropped = img[y:y+h, x:x+w]

        # Save the cropped image as a new JPEG file
        cv2.imwrite(output_path, img_cropped)

In [62]:
def georeference(input_path, output_path, xmin, xmax, ymin, ymax):
    # Load the input image
    input_image = gdal.Open(input_path)

    # Get the image dimensions
    image_width = input_image.RasterXSize
    image_height = input_image.RasterYSize
    num_bands = input_image.RasterCount

    # Create a spatial reference for the output GeoTIFF
    output_srs = osr.SpatialReference()
    output_srs.ImportFromEPSG(4326)  # Use WGS84 coordinate system

    # Create the output GeoTIFF file
    driver = gdal.GetDriverByName('GTiff')
    output_image = driver.Create(
    output_path, 
    image_width, 
    image_height, 
    num_bands, 
    gdal.GDT_Byte
    )

    # Write the output image data
    for band_index in range(1, num_bands + 1):
        input_band = input_image.GetRasterBand(band_index)
        output_band = output_image.GetRasterBand(band_index)
        output_band.WriteArray(input_band.ReadAsArray())

    # Set the output image geotransform
    geotransform = (
        xmin, 
        (xmax - xmin) / image_width, 
        0, 
        ymax, 
        0, 
        (ymin - ymax) / image_height
    )
    output_image.SetGeoTransform(geotransform)

    # Set the output image projection
    output_image.SetProjection(output_srs.ExportToWkt())

    # Close the input and output images
    input_image = None
    output_image = None

In [79]:
if not os.path.exists(TEMP_CROP):
    os.makedirs(TEMP_CROP)
  
if not os.path.exists(TEMP_ROI):
    os.makedirs(TEMP_ROI)

if not os.path.exists(RESULT):
    os.makedirs(RESULT)

In [99]:
list_peta = os.listdir(SOURCE)
df = pd.read_csv(COORD, dtype={'idsls': str})

In [105]:
for i in tqdm(list_peta):
    name = i.split('_')[0]
    name_ext = i.split('.')[0]

    # file_path = 'rename/{}_WS.jpg'.format(name) 
    # angel = compute_skew(file_path)
    # dst = deskew(file_path,angel)
    # cv2.imwrite('temp_file/crop/{}_WS.jpg'.format(name), dst)

    try:
        get_loc(
            '{}/{}'.format(SOURCE, i),
            '{}/{}'.format(TEMP_ROI, i)
        )
    
        crop_map(
            '{}/{}'.format(TEMP_ROI, i),
            '{}/{}'.format(TEMP_CROP, i)
        )

        xmin, xmax, ymin, ymax = df.loc[
            df['idsls']==name, 
            ['xmin', 'xmax', 'ymin', 'ymax']
        ].values[0]
        # print(xmin, xmax, ymin, ymax)
    except:
        print('{}/{}'.format(TEMP_ROI, i))

    try:
        georeference(
            '{}/{}'.format(TEMP_CROP, i), 
            '{}/{}.tif'.format(RESULT, name_ext),
            xmin, xmax, ymin, ymax
        )
    except:
        pass

# shutil.rmtree('RESULT')

 22%|████████████████▊                                                            | 759/3486 [20:36<2:21:37,  3.12s/it]

/Terry/georeference/temp_file/ROI/52060300030006.jpg


 48%|█████████████████████████████████████▏                                        | 1662/3486 [44:21<36:17,  1.19s/it]

/Terry/georeference/temp_file/ROI/52060510110008.png


 59%|█████████████████████████████████████████████▊                                | 2045/3486 [52:52<28:03,  1.17s/it]

/Terry/georeference/temp_file/ROI/520606002020004_WS.png


100%|███████████████████████████████████████████████████████████████████████████▉| 3485/3486 [1:22:31<00:01,  1.24s/it]

/Terry/georeference/temp_file/ROI/525060410070008_WS.png


100%|████████████████████████████████████████████████████████████████████████████| 3486/3486 [1:22:33<00:00,  1.42s/it]
